In [ ]:
import numpy as np
from scipy.optimize import curve_fit, fsolve
import matplotlib.pyplot as plt
from scipy.constants import k, e

In [ ]:
DATA_DIR = "../dataset/Data_1k_sets/Data_1k_rng1/"

Input = np.loadtxt(DATA_DIR + 'LHS_parameters_m.txt', delimiter=',') # input [n_samples, 31]
Output = np.loadtxt(DATA_DIR + 'iV_m.txt', delimiter=',') # output [n_samples, n_points = len(Va)]
Va = np.concatenate((np.arange(0, 0.41, 0.1), np.arange(0.425, 1.401, 0.025)))
T_cell_K = 298.15
V_t = k * T_cell_K / e

I_exp = Output[1, :]
V_exp = Va

## Initial Experiment with fsolve

Sounds promising, until you realised for reconstruction it worse really bad if you don't have I exp (which is cheating lol)

In [ ]:
class SingleDiodeModel:
    """
    Single diode model for solar cells.
    """
    def __init__(self, T_cell_K=298.15, V_oc_approx=0.6, experimental_I_for_guesses=None, param_bounds=None):
        self.V_t = k * T_cell_K / e
        self.experimental_I_for_guesses = experimental_I_for_guesses
        self.V_oc_approx = V_oc_approx
        self.param_lower_bounds = param_bounds[0] if param_bounds else None

    def _diode_equation_solver(self, I, V, I_ph, I_0, n, R_s, R_sh):
        return I_ph - I_0 * (np.exp((V + I * R_s) / (n * self.V_t)) - 1) - (V + I * R_s) / R_sh - I

    def __call__(self, V_array, I_ph, I_0, n, R_s, R_sh):
        I_calculated = np.zeros_like(V_array, dtype=float)
        
        # Sort V_array to ensure smooth guessing from one point to the next
        # If V_array is not sorted, this might lead to jumps in guesses.
        # However, for IV curves, V_array is typically monotonically increasing.
        
        for i, V in enumerate(V_array):
            if self.experimental_I_for_guesses is not None and i < len(self.experimental_I_for_guesses):
                I_guess = self.experimental_I_for_guesses[i]
            else:
                # Improved initial guess strategy
                if V < 0.1 * self.V_oc_approx: # Near short circuit
                    I_guess = I_ph # Start with photocurrent
                elif V > 0.9 * self.V_oc_approx: # Near open circuit
                    I_guess = 0.0 # Current is close to zero
                else:
                    # A more general guess for the main part of the curve.
                    # This approximates the current by neglecting R_s and R_sh for the guess.
                    # It's an explicit form of the diode equation without series/shunt resistances.
                    # This is still not perfect but significantly better than a linear guess.
                    I_guess = I_ph - I_0 * (np.exp(V / (n * self.V_t)) - 1)
                    
                    # Ensure guess is physically reasonable (e.g., not negative)
                    I_guess = max(0.0, I_guess) 
                    
                # You can also use the previous calculated point as a guess for the next
                # if iterating through V_array in a monotonic order.
                if i > 0:
                    I_guess = (I_guess + I_calculated[i-1]) / 2.0 # Average with previous to smooth it out
                
            sol = fsolve(
                self._diode_equation_solver,
                I_guess,
                args=(V, I_ph, I_0, n, R_s, R_sh),
                xtol=1.49012e-08,
                maxfev=1000 # Increase max function evaluations if convergence issues
            )
            I_calculated[i] = sol[0]
            
            # Post-process: Ensure current doesn't go significantly negative unphysically
            if I_calculated[i] < -1e-6: # Allow for small numerical noise
                I_calculated[i] = 0.0
                
        return I_calculated

In [ ]:
# Define bounds for curve_fit based on the measured short-circuit current
I_sc_approx = I_exp[0] # Isc is the first value in the I_exp array
bounds_for_fitting = (
    [0.5 * I_sc_approx, 1e-12, 0.8, 1e-4, 10],   # Lower bounds
    [1.5 * I_sc_approx, 1e-6, 2.5, 0.5, 5000]    # Upper bounds
)
V_oc_idx = np.where(I_exp < 0.01 * I_sc_approx)[0]
V_oc_approx = V_exp[V_oc_idx[0]] if len(V_oc_idx) > 0 else 0.62

model_to_fit = SingleDiodeModel(
    V_oc_approx=V_oc_approx, 
    experimental_I_for_guesses=I_exp, 
    param_bounds=bounds_for_fitting
)

# less efficient way just randomly choose initial values for Rs and Rsh
# p0 = [
#     I_sc_approx,
#     2e-9,
#     1.4,
#     0.05,
#     300
# ]

# Initial guesses for the parameters based on the experimental data
# NOTE: This is a really good optimization for runtime since it turns 3-4 seconds into 0.3 seconds
Rs_guess = - (V_exp[1] - V_exp[0]) / (I_exp[1] - I_exp[0] + 1e-12) if len(I_exp) > 1 else 0.05
Rsh_guess = - (V_exp[-1] - V_exp[-2]) / (I_exp[-1] - I_exp[-2] + 1e-12) if len(I_exp) > 1 else 300
p0 = [
    I_sc_approx,
    2e-9,
    1.2,
    Rs_guess,
    Rsh_guess
]

# Ensure p0 is within bounds
for i in range(len(p0)):
    p0[i] = max(p0[i], bounds_for_fitting[0][i])
    p0[i] = min(p0[i], bounds_for_fitting[1][i])

print(f"\nInitial guesses (p0): {[f'{x:.2e}' for x in p0]}")
print(f"Parameter bounds (lower): {[f'{x:.2e}' for x in bounds_for_fitting[0]]}")
print(f"Parameter bounds (upper): {[f'{x:.2e}' for x in bounds_for_fitting[1]]}")

try:
    import time
    start_time = time.time()
    popt, pcov = curve_fit(
        model_to_fit, V_exp, I_exp, p0=p0, bounds=bounds_for_fitting, method='trf', max_nfev=400,
    )
    end_time = time.time()
    print(f"\nCurve fitting completed in {end_time - start_time:.2f} seconds.")
except (RuntimeError, ValueError) as e:
    print(f"\nError during curve_fit: {e}")
    print("Optimization failed. Check initial guesses, bounds, or data quality.")
else:
    I_ph_fit, I_0_fit, n_fit, R_s_fit, R_sh_fit = popt

    print("\n--- Extracted Parameters ---")
    print(f"I_ph (Photocurrent):     {I_ph_fit:.4f} A")
    print(f"I_0 (Saturation Current):{I_0_fit:.3e} A")
    print(f"n (Ideality Factor):     {n_fit:.3f}")
    print(f"R_s (Series Resistance): {R_s_fit:.4f} Ohms")
    print(f"R_sh (Shunt Resistance): {R_sh_fit:.2f} Ohms")

    I_fit = model_to_fit(V_exp, I_ph_fit, I_0_fit, n_fit, R_s_fit, R_sh_fit)

    plt.figure(figsize=(10, 7))
    plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
    if not np.any(np.isinf(I_fit)) and not np.any(np.isnan(I_fit)):
        plt.plot(V_exp, I_fit, 'r-', label='Fitted Single Diode Model')
    else:
        print("Fit resulted in Inf/NaN values, cannot plot fitted curve accurately.")

    plt.xlabel('Voltage (V)')
    plt.ylabel('Current (A)')
    plt.title('Solar Cell I-V Curve: Experimental vs. Fitted Model')
    plt.legend()
    plt.grid(True)
    plt.ylim(bottom=max(0, np.min(I_exp)-0.1*I_sc_approx), 
             top=1.1*I_sc_approx)
    plt.show()

    if not np.any(np.isinf(I_fit)) and not np.any(np.isnan(I_fit)):
        rmse = np.sqrt(np.mean((I_exp - I_fit)**2))
        print(f"\nRoot Mean Squared Error (RMSE): {rmse:.4e} A")
    else:
        print("\nCould not calculate RMSE due to invalid fitted current values.")


Now there is an issue in inference we cannot simply take the 5 parameters and create the IV curve, because without having the I_exp as starting points for fsolve it is really inefficient...

The following plot shows when reconstructing it with the params

In [ ]:
reconstruct_model = SingleDiodeModel(
    T_cell_K=298.15,
    V_oc_approx=V_oc_approx,
    experimental_I_for_guesses=None,
    param_bounds=None
)

start = time.time()
# take the fitted parameters (which will be predicted by the model)
I_fit = reconstruct_model(V_exp, I_ph_fit, I_0_fit, n_fit, R_s_fit, R_sh_fit)
print(f"\nReconstruction completed in {time.time() - start:.2f} seconds.")

plt.figure(figsize=(10, 7))
plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
plt.plot(V_exp, I_fit, 'r-', label='Reconstructed Single Diode Model')
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('Reconstructed Solar Cell I-V Curve')
plt.legend()
plt.grid(True)
plt.ylim(bottom=max(0, np.min(I_exp)-0.1*I_sc_approx), 
         top=1.1*I_sc_approx)
plt.show()

## Experiments with Pvlib solver instead of fsolve

Now that we've given up finding out a solver ourselves, I can just steal this algorithm from some random library actually! Essentially we use their solver that takes a bunch of V points and the 5 parameters to return the I points. And then we use scipy to fit the 5 parameters to the I points.

In [ ]:
from pvlib.pvsystem import calcparams_desoto

I_ph_fit, I_0_fit, R_s_fit, R_sh_fit, n_fit = calcparams_desoto(
    effective_irradiance=1000,  # W/m^2
    temp_cell=T_cell_K - 273.15,  # Convert K to C
    alpha_sc=0.0005,  # Temperature coefficient of short-circuit current
    a_ref=1.2,  # Diode ideality factor
    I_L_ref=I_ph_fit,  # Light-generated current
    I_o_ref=I_0_fit,  # Saturation current
    R_s=R_s_fit,  # Series resistance
    R_sh_ref=R_sh_fit,  # Shunt resistance
)

In [ ]:
from pvlib.pvsystem import singlediode

result = singlediode(
    I_ph_fit, I_0_fit, R_s_fit, R_sh_fit, n_fit * V_t,
    # ivcurve_pnts = len(V_exp), # number of points in the I-V curve
)

i_sc = result['i_sc']
v_oc = result['v_oc']
i_mp = result['i_mp']
v_mp = result['v_mp']
p_mp = result['p_mp']
i_x = result['i_x'] # current at V = 0.5 * V_oc
i_xx = result['i_xx'] # current at V = 0.5 * (V_oc + V_mp)
# i_lambert = result['i']
# v_lambert = result['v']

# plot both experimental and points from the pvlib model
plt.figure(figsize=(10, 7))
plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
# Plot points from pvlib model
# plt.plot([0, v_mp, v_oc], [i_sc, i_mp, 0], 'r*', label='Key Points (pvlib)', markersize=10)
# plt.plot([0.5*v_oc, 0.5*(v_oc + v_mp)], [i_x, i_xx], 'g*', label='Additional Points (pvlib)', markersize=10)
# plt.plot(v_lambert, i_lambert, 'm-', label='Lambertian Fit (pvlib)')
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('Reconstructed Solar Cell I-V Curve')
plt.legend()
plt.grid(True)
plt.ylim(bottom=max(0, np.min(I_exp)-0.1*I_sc_approx), 
         top=1.1*I_sc_approx)
plt.show()

In [ ]:
from pvlib.singlediode import bishop88_i_from_v

p0 = [
    10,
    1.2e-7,
    0.32,
    300,
    1.5 * V_t,
]

# calculate current for every voltage point
# I_bishop = bishop88_i_from_v(
#     V_exp, I_ph_fit, I_0_fit, R_s_fit, R_sh_fit, n_fit * V_t
# )

I_bishop = bishop88_i_from_v(
    V_exp, *p0,
)

plt.figure(figsize=(10, 7))
plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
plt.plot(V_exp, I_bishop, 'r-', label='Bishop 88 Model')
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('Bishop 88 Model I-V Curve')
plt.legend()
plt.grid(True)
# plt.ylim(bottom=max(0, np.min(I_exp)-0.1*I_sc_approx), 
#          top=1.1*I_sc_approx)
plt.show()

**THIS SOUNDS TO BE A PROMISING METHOD!!** Try to keep investigating this and see how to make it work. This is literally just curve fitting to get the five parameters, no way it does not work...

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.constants import k, e
from pvlib.singlediode import bishop88_i_from_v

class SingleDiodeModelFitter:
    """
    A class to encapsulate the single diode model and its parameter fitting.
    Uses pvlib's bishop88_i_from_v for robust IV curve calculation.
    """
    def __init__(self, T_cell_K=298.15):
        self.T_cell_K = T_cell_K
        self.V_t = k * self.T_cell_K / e # Thermal voltage (kT/q)

    def _objective_function(self, params, V_exp, I_exp):
        """Calculates sum of squared errors between experimental and predicted current."""
        I_ph, I_0, n, R_s, R_sh = params

        # Ensure parameters are within reasonable physical bounds
        I_ph = max(1e-6, I_ph)
        I_0 = max(1e-15, I_0)
        n = max(1.0, n)
        R_s = max(0.0, R_s)
        R_sh = max(1e-3, R_sh)

        try:
            I_predicted = bishop88_i_from_v(V_exp, I_ph, I_0, R_s, R_sh, n * self.V_t)
        except Exception:
            return 1e12 # Return large error on solver failure

        return np.sum((I_exp - I_predicted)**2)

    def fit_parameters(self, V_exp, I_exp, initial_guess=None, bounds=None):
        """
        Fits the 5 single diode model parameters to experimental IV data.
        
        Args:
            V_exp (np.array): Experimental voltage data.
            I_exp (np.array): Experimental current data.
            initial_guess (list, optional): Initial guess for [I_ph, I_0, n, R_s, R_sh].
            bounds (list of tuples, optional): Bounds for each parameter.
                                                
        Returns:
            dict: 'success', 'message', and 'fitted_params'.
        """
        if initial_guess is None:
            idx_isc = np.argmin(np.abs(V_exp))
            I_ph_guess = I_exp[idx_isc] if I_exp[idx_isc] > 0 else np.max(I_exp) * 1.05
            initial_guess = [I_ph_guess, 1e-6, 1.5, 0.1, 1000.0]

        if bounds is None:
            bounds = [
                (0.5 * initial_guess[0], 1.5 * initial_guess[0]),
                (1e-15, 1e-3),
                (1.0, 2.0),
                (0.0, 5.0),
                (10.0, 1e5)
            ]

        result = minimize(
            self._objective_function,
            initial_guess,
            args=(V_exp, I_exp),
            method='L-BFGS-B',
            bounds=bounds,
            options={'disp': False, 'maxiter': 1000}
        )

        if result.success:
            return {'success': True, 'message': result.message, 'fitted_params': result.x}
        else:
            return {'success': False, 'message': result.message, 'fitted_params': None}

    def reconstruct_curve(self, V_array, fitted_params):
        """Reconstructs the IV curve using the fitted parameters."""
        I_ph, I_0, n, R_s, R_sh = fitted_params
        return bishop88_i_from_v(V_array, I_ph, I_0, R_s, R_sh, n * self.V_t)


In [ ]:
# use the above code to fit the parameters
fitter = SingleDiodeModelFitter()
result = fitter.fit_parameters(V_exp, I_exp)
if result['success']:
    fitted_params = result['fitted_params']
    print("\n--- Fitted Parameters ---")
    print(f"I_ph (Photocurrent):     {fitted_params[0]:.4f} A")
    print(f"I_0 (Saturation Current):{fitted_params[1]:.3e} A")
    print(f"n (Ideality Factor):     {fitted_params[2]:.3f}")
    print(f"R_s (Series Resistance): {fitted_params[3]:.4f} Ohms")
    print(f"R_sh (Shunt Resistance): {fitted_params[4]:.2f} Ohms")

    I_fit = fitter.reconstruct_curve(V_exp, fitted_params)

    plt.figure(figsize=(10, 7))
    plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
    plt.plot(V_exp, I_fit, 'r-', label='Fitted Single Diode Model')
    plt.xlabel('Voltage (V)')
    plt.ylabel('Current (A)')
    plt.title('Fitted Single Diode Model I-V Curve')
    plt.legend()
    plt.grid(True)
    plt.ylim(bottom=max(0, np.min(I_exp)-0.1*I_sc_approx), 
             top=1.1*I_sc_approx)
    plt.show()
    rmse = np.sqrt(np.mean((I_exp - I_fit)**2))
    print(f"\nRoot Mean Squared Error (RMSE): {rmse:.4e} A")
else:
    print("\nFitting failed.")
    print(f"Message: {result['message']}")
    print("Check initial guesses, bounds, or data quality.")

## Other failed attempts

We tried using this (exactly same usage, just different names) in the above code because it is said that using Jacobian is better for fsolve to converge, but it is not working (same result as the above SDM model)...

In [ ]:
class SingleDiodeModelWithJacobian:
    """
    Single diode model for solar cells.
    """
    def __init__(self, T_cell_K=298.15, V_oc_approx=0.6, experimental_I_for_guesses=None, param_bounds=None):
        self.V_t = k * T_cell_K / e
        self.experimental_I_for_guesses = experimental_I_for_guesses
        self.V_oc_approx = V_oc_approx
        self.param_lower_bounds = param_bounds[0] if param_bounds else None

    def _diode_equation_solver(self, I, V, I_ph, I_0, n, R_s, R_sh):
        return I_ph - I_0 * (np.exp((V + I * R_s) / (n * self.V_t)) - 1) - (V + I * R_s) / R_sh - I

    def _diode_equation_jacobian(self, I, V, I_ph, I_0, n, R_s, R_sh):
        # Derivative of _diode_equation_solver with respect to I
        return -I_0 * (R_s / (n * self.V_t)) * np.exp((V + I * R_s) / (n * self.V_t)) - R_s / R_sh - 1

    def __call__(self, V_array, I_ph, I_0, n, R_s, R_sh):
        I_calculated = np.zeros_like(V_array, dtype=float)
        
        # Initialize I_guess for the first point
        I_previous_guess = I_ph # Good starting guess for the first point near Isc

        for i, V in enumerate(V_array):
            if self.experimental_I_for_guesses is not None and i < len(self.experimental_I_for_guesses):
                I_guess = self.experimental_I_for_guesses[i]
            else:
                # Use a combined strategy: simplified model for general guess,
                # and previous point for continuity.
                # Simplified guess (neglecting R_s, R_sh for first approximation)
                simplified_I_guess = I_ph - I_0 * (np.exp(V / (n * self.V_t)) - 1)
                simplified_I_guess = max(0.0, simplified_I_guess) # Ensure non-negative

                # If this is the first point, use the simplified guess.
                # Otherwise, average with the previously converged solution for smoother guessing.
                if i == 0:
                    I_guess = simplified_I_guess
                else:
                    I_guess = (simplified_I_guess + I_calculated[i-1]) / 2.0
                
                # A fallback to I_ph if the current V is very low (near short circuit)
                if V < 0.05 * self.V_oc_approx:
                    I_guess = I_ph

            sol = fsolve(
                self._diode_equation_solver,
                I_guess,
                args=(V, I_ph, I_0, n, R_s, R_sh),
                fprime=self._diode_equation_jacobian, # Provide the Jacobian
                xtol=1.49012e-08,
                maxfev=1000
            )
            I_calculated[i] = sol[0]
            
            # Ensure current doesn't go significantly negative
            if I_calculated[i] < -1e-6:
                I_calculated[i] = 0.0
                
        return I_calculated

So we thought we can just use a Lambert W function to solve it analytically with parameters obtained from the fit using the fsolve method. However, this does not seem promising...

In [ ]:
from scipy.constants import k, e
from scipy.special import lambertw

class SingleDiodeModelLambertW:
    def __init__(self, T_cell_K=298.15):
        self.T_cell_K = T_cell_K

    def __call__(self, V_array, I_ph, I_0, n, R_s, R_sh):
        V_t = n * k * self.T_cell_K / e
        V = np.asarray(V_array)
        exp_arg = (V + (I_ph + I_0) * R_s) / (n * V_t)
        exp_arg = np.clip(exp_arg, -100, 100)  # Prevent overflow
        argW = (I_ph + I_0) * R_sh * np.exp(exp_arg) / (I_0 * R_sh)
        I = (
            I_ph
            - n * V_t / R_s * lambertw(argW).real
            - (V + n * V_t * lambertw(argW).real) / R_sh
        )
        return I


In [ ]:
# popt = [I_ph, I_0, n, R_s, R_sh] from fitting
print("Fitted parameters:", popt)

# Check for invalid or extreme values
param_names = ["I_ph", "I_0", "n", "R_s", "R_sh"]
for name, val in zip(param_names, popt):
    print(f"{name}: {val}")

# Replace invalid values with small positive numbers if needed
popt_checked = [
    max(val, 1e-12) if i in [1, 3, 4] else val  # I_0, R_s, R_sh must be > 0
    for i, val in enumerate(popt)
]
print("Sanitized parameters:", popt_checked)

# Now use Lambert W model for inference
sdm_lw = SingleDiodeModelLambertW()
I_lw = sdm_lw(V_exp, *popt_checked)

plt.figure(figsize=(10, 7))
plt.plot(V_exp, I_exp, 'bo', label='Experimental Data', markersize=5)
plt.plot(V_exp, I_lw, 'g--', label='Lambert W Inference (sanitized)', alpha=0.7)
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('IV Curve: Lambert W Inference from Fitted Parameters (Sanitized)')
plt.legend()
plt.grid(True)
plt.show()
